In [2]:
from google.colab import drive
drive.mount('/content/drive')
!pwd

Mounted at /content/drive
/content


In [3]:
!ls
%cd drive/MyDrive/DA/clustering
!pwd

drive  sample_data
/content/drive/MyDrive/DA/clustering
/content/drive/MyDrive/DA/clustering


In [4]:
import numpy as np
import pandas as pd

####Load dataset

In [119]:
df_data_org = pd.read_csv('football_data.csv', na_values=np.NaN)
print(df_data_org.shape)
display(df_data_org.head())
display(df_data_org.info())
display(df_data_org.describe())

(18207, 89)


,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,...,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,€110.5M,€565K,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,10.0,"Jul 1, 2004",NaN,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,...,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€77M,€405K,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,7.0,"Jul 10, 2018",NaN,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,...,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,€118.5M,€290K,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,10.0,"Aug 3, 2017",NaN,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,...,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,1.0,"Jul 1, 2011",NaN,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,€102M,€355K,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,7.0,"Aug 30, 2015",NaN,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,...,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18207 entries, 0 to 18206
Data columns (total 89 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                18207 non-null  int64  
 1   ID                        18207 non-null  int64  
 2   Name                      18207 non-null  object 
 3   Age                       18207 non-null  int64  
 4   Photo                     18207 non-null  object 
 5   Nationality               18207 non-null  object 
 6   Flag                      18207 non-null  object 
 7   Overall                   18207 non-null  int64  
 8   Potential                 18207 non-null  int64  
 9   Club                      17966 non-null  object 
 10  Club Logo                 18207 non-null  object 
 11  Value                     18207 non-null  object 
 12  Wage                      18207 non-null  object 
 13  Special                   18207 non-null  int64  
 14  Prefer

None

,Unnamed: 0,ID,Age,Overall,Potential,Special,International Reputation,Weak Foot,Skill Moves,Jersey Number,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
count,18207.000000,18207.000000,18207.000000,18207.000000,18207.000000,18207.000000,18159.000000,18159.000000,18159.000000,18147.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000
mean,9103.000000,214298.338606,25.122206,66.238699,71.307299,1597.809908,1.113222,2.947299,2.361308,19.546096,49.734181,45.550911,52.298144,58.686712,42.909026,55.371001,47.170824,42.863153,52.711933,58.369459,64.614076,64.726967,63.503607,61.836610,63.966573,55.460047,65.089432,63.219946,65.311967,47.109973,55.868991,46.698276,49.958478,53.400903,48.548598,58.648274,47.281623,47.697836,45.661435,16.616223,16.391596,16.232061,16.388898,16.710887
std,5256.052511,29965.244204,4.669943,6.908930,6.136496,272.586016,0.394031,0.660456,0.756164,15.947765,18.364524,19.525820,17.379909,14.699495,17.694408,18.910371,18.395264,17.478763,15.327870,16.686595,14.927780,14.649953,14.766049,9.010464,14.136166,17.237958,11.820044,15.894741,12.557000,19.260524,17.367967,20.696909,19.529036,14.146881,15.704053,11.436133,19.904397,21.664004,21.289135,17.695349,16.906900,16.502864,17.034669,17.955119
min,0.000000,16.000000,16.000000,46.000000,48.000000,731.000000,1.000000,1.000000,1.000000,1.000000,5.000000,2.000000,4.000000,7.000000,4.000000,4.000000,6.000000,3.000000,9.000000,5.000000,12.000000,12.000000,14.000000,21.000000,16.000000,2.000000,15.000000,12.000000,17.000000,3.000000,11.000000,3.000000,2.000000,10.000000,5.000000,3.000000,3.000000,2.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,4551.500000,200315.500000,21.000000,62.000000,67.000000,1457.000000,1.000000,3.000000,2.000000,8.000000,38.000000,30.000000,44.000000,54.000000,30.000000,49.000000,34.000000,31.000000,43.000000,54.000000,57.000000,57.000000,55.000000,56.000000,56.000000,45.000000,58.000000,56.000000,58.000000,33.000000,44.000000,26.000000,38.000000,44.000000,39.000000,51.000000,30.000000,27.000000,24.000000,8.000000,8.000000,8.000000,8.000000,8.000000
50%,9103.000000,221759.000000,25.000000,66.000000,71.000000,1635.000000,1.000000,3.000000,2.000000,17.000000,54.000000,49.000000,56.000000,62.000000,44.000000,61.000000,48.000000,41.000000,56.000000,63.000000,67.000000,67.000000,66.000000,62.000000,66.000000,59.000000,66.000000,66.000000,67.000000,51.000000,59.000000,52.000000,55.000000,55.000000,49.000000,60.000000,53.000000,55.000000,52.000000,11.000000,11.000000,11.000000,11.000000,11.000000
75%,13654.500000,236529.500000,28.000000,71.000000,75.000000,1787.000000,1.000000,3.000000,3.000000,26.000000,64.000000,62.000000,64.000000,68.000000,57.000000,68.000000,62.000000,57.000000,64.000000,69.000000,75.000000,75.000000,74.000000,68.000000,74.000000,68.000000,73.000000,74.000000,74.000000,62.000000,69.000000,64.000000,64.000000,64.000000,60.000000,67.000000,64.000000,66.000000,64.000000,14.000000,14.000000,14.000000,14.000000,14.000000
max,18206.000000,246620.000000,45.000000,94.000000,95.000000,2346.000000,5.000000,5.000000,5.000000,99.000000,93.000000,95.000000,94.000000,93.000000,90.000000,97.000000,94.000000,94.000000,93.000000,96.000000,97.000000,96.000000,96.000000,96.000000,96.000000,95.000000,95.000000,96.000000,97.000000,94.000

####Drop rows having no position info... as these cols also do not have values for many other attributes


In [120]:
df_data = df_data_org[df_data_org['Position'].notna()]
print(df_data.shape)

(18147, 89)


####Drop nonrequired cols

In [121]:
df_data = df_data.drop(['Unnamed: 0','ID','Name','Photo','Nationality','Flag','Club','Club Logo','Joined','Loaned From','Contract Valid Until'], axis=1)
print(df_data.shape)

(18147, 78)


####Convert currency columns to float

In [122]:
df_data['Value'] = df_data['Value'].apply(lambda c: float(c[1:-1]) if (pd.notnull(c) and (c[-1]=='M' or c[-1]=='K')) else 0.0)
print(df_data['Value'].shape)

df_data['Wage'] = df_data['Wage'].apply(lambda c: float(c[1:-1]) if (pd.notnull(c) and (c[-1]=='M' or c[-1]=='K')) else 0.0)
print(df_data['Wage'].shape)

(18147,)
(18147,)


####Convert real face to binary values

In [123]:
print(df_data['Real Face'].unique())
df_data['Real Face'] = df_data['Real Face'].apply(lambda c: 1 if c=='Yes' else 0)
print(df_data['Real Face'].shape)
print(df_data['Real Face'].unique())

['Yes' 'No']
(18147,)
[1 0]


####Convert body type to numeric ordinal values

In [124]:
print(len(df_data[df_data['Body Type']=='Normal']))
print(len(df_data[df_data['Body Type']=='Stocky']))
print(len(df_data[df_data['Body Type']=='Lean']))

df_data['Body Type'] = df_data['Body Type'].apply(lambda c: 'Normal' if c not in ['Normal','Stocky','Lean'] else c)
print(df_data['Body Type'].unique())

cat_to_num = {"Body Type":{'Lean':0, 'Normal':1, 'Stocky':2}}
df_data = df_data.replace(cat_to_num)
print(df_data['Body Type'].unique())

10588
1140
6412
['Normal' 'Lean' 'Stocky']
[1 0 2]


####Convert work rate to ordinal values

In [125]:
print(df_data['Work Rate'].unique())
df_data['Attack Work Rate'] = df_data['Work Rate'].apply(lambda c: c.split('/ ')[0])
df_data['Defense Work Rate'] = df_data['Work Rate'].apply(lambda c: c.split('/ ')[1])
print(df_data['Attack Work Rate'].unique())
print(df_data['Defense Work Rate'].unique())
cat_to_num = {"Attack Work Rate":{'Low':0, 'Medium':1, 'High':2}, "Defense Work Rate":{'Low':0, 'Medium':1, 'High':2}}
df_data = df_data.replace(cat_to_num)
print(df_data['Attack Work Rate'].unique())
print(df_data['Defense Work Rate'].unique())
df_data = df_data.drop(['Work Rate'], axis=1)

['Medium/ Medium' 'High/ Low' 'High/ Medium' 'High/ High' 'Medium/ High'
 'Medium/ Low' 'Low/ High' 'Low/ Medium' 'Low/ Low']
['Medium' 'High' 'Low']
['Medium' 'Low' 'High']
[1 2 0]
[1 0 2]


####Convert preferred foot to binary

In [126]:
print(df_data['Preferred Foot'].unique())
df_data['Preferred Foot'] = df_data['Preferred Foot'].apply(lambda c: 1 if c=='Right' else 0)
print(df_data['Preferred Foot'].shape)
print(df_data['Preferred Foot'].unique())

['Left' 'Right']
(18147,)
[0 1]


####Convert height to float

In [127]:
print(df_data['Height'].unique())
df_data['Height'] = df_data['Height'].apply(lambda c: float(c[0]+'.'+c[2]))
print(df_data['Height'].unique())

["5'7" "6'2" "5'9" "6'4" "5'11" "5'8" "6'0" "5'6" "5'10" "6'6" "6'1" "5'4"
 "6'3" "5'5" "6'5" "6'7" "5'3" "5'2" "6'8" "5'1" "6'9"]
[5.7 6.2 5.9 6.4 5.1 5.8 6.  5.6 6.6 6.1 5.4 6.3 5.5 6.5 6.7 5.3 5.2 6.8
 6.9]


####Convert weight to float

In [128]:
print(df_data['Weight'].unique())
df_data['Weight'] = df_data['Weight'].apply(lambda c: int(c[:-3]))
print(df_data['Weight'].unique())

['159lbs' '183lbs' '150lbs' '168lbs' '154lbs' '163lbs' '146lbs' '190lbs'
 '181lbs' '192lbs' '176lbs' '172lbs' '148lbs' '165lbs' '196lbs' '161lbs'
 '187lbs' '212lbs' '170lbs' '203lbs' '157lbs' '185lbs' '130lbs' '174lbs'
 '207lbs' '134lbs' '141lbs' '152lbs' '179lbs' '132lbs' '201lbs' '198lbs'
 '209lbs' '214lbs' '143lbs' '205lbs' '137lbs' '194lbs' '216lbs' '139lbs'
 '220lbs' '126lbs' '218lbs' '123lbs' '227lbs' '128lbs' '223lbs' '225lbs'
 '121lbs' '115lbs' '117lbs' '236lbs' '229lbs' '243lbs' '110lbs' '119lbs'
 '234lbs']
[159 183 150 168 154 163 146 190 181 192 176 172 148 165 196 161 187 212
 170 203 157 185 130 174 207 134 141 152 179 132 201 198 209 214 143 205
 137 194 216 139 220 126 218 123 227 128 223 225 121 115 117 236 229 243
 110 119 234]


####Convert release clause to float

In [129]:
df_data['Release Clause'] = df_data['Release Clause'].apply(lambda c: float(c[1:-1]) if (pd.notnull(c) and (c[-1]=='M' or c[-1]=='K')) else 0.0)
df_data['Release Clause'].fillna(df_data['Release Clause'].median(), inplace = True)
print(df_data['Release Clause'].shape)
print(df_data['Release Clause'].unique())


(18147,)
[226.5 127.1 228.1 ...  67.  101.  147. ]


####Convert positional scores to float

In [130]:
scores = ['LS','ST','RS','LW','LF','CF','RF','RW','LAM','CAM','RAM','LM','LCM','CM','RCM','RM','LWB','LDM','CDM','RDM','RWB','LB','LCB','CB','RCB','RB']
for score in scores:
    df_data[score] = df_data[score].apply(lambda c: float(c.split('+')[0]) if pd.notnull(c) else 0.0)
    df_data[score].fillna(df_data[score].median(), inplace = True)
    print(score,df_data[score].shape,df_data[score][:3],'\n')  

df_data = df_data.drop(['Position'], axis=1)

LS (18147,) 0    88.0
1    91.0
2    84.0
Name: LS, dtype: float64 

ST (18147,) 0    88.0
1    91.0
2    84.0
Name: ST, dtype: float64 

RS (18147,) 0    88.0
1    91.0
2    84.0
Name: RS, dtype: float64 

LW (18147,) 0    92.0
1    89.0
2    89.0
Name: LW, dtype: float64 

LF (18147,) 0    93.0
1    90.0
2    89.0
Name: LF, dtype: float64 

CF (18147,) 0    93.0
1    90.0
2    89.0
Name: CF, dtype: float64 

RF (18147,) 0    93.0
1    90.0
2    89.0
Name: RF, dtype: float64 

RW (18147,) 0    92.0
1    89.0
2    89.0
Name: RW, dtype: float64 

LAM (18147,) 0    93.0
1    88.0
2    89.0
Name: LAM, dtype: float64 

CAM (18147,) 0    93.0
1    88.0
2    89.0
Name: CAM, dtype: float64 

RAM (18147,) 0    93.0
1    88.0
2    89.0
Name: RAM, dtype: float64 

LM (18147,) 0    91.0
1    88.0
2    88.0
Name: LM, dtype: float64 

LCM (18147,) 0    84.0
1    81.0
2    81.0
Name: LCM, dtype: float64 

CM (18147,) 0    84.0
1    81.0
2    81.0
Name: CM, dtype: float64 

RCM (18147,) 0    84.0
1  

####Set nan values to median of resp col

In [134]:
for col in df_data.columns:
    df_data[col].fillna(df_data[col].median(), inplace=True)

display(df_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18147 entries, 0 to 18206
Data columns (total 78 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       18147 non-null  int64  
 1   Overall                   18147 non-null  int64  
 2   Potential                 18147 non-null  int64  
 3   Value                     18147 non-null  float64
 4   Wage                      18147 non-null  float64
 5   Special                   18147 non-null  int64  
 6   Preferred Foot            18147 non-null  int64  
 7   International Reputation  18147 non-null  float64
 8   Weak Foot                 18147 non-null  float64
 9   Skill Moves               18147 non-null  float64
 10  Body Type                 18147 non-null  int64  
 11  Real Face                 18147 non-null  int64  
 12  Jersey Number             18147 non-null  float64
 13  Height                    18147 non-null  float64
 14  Weight

None

####Save the processed dataframe to csv

In [135]:
df_data.to_csv('processed_football_data.csv', index=False)